In [17]:
# from pyforest import * 
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn import metrics
from sklearn.cluster import DBSCAN, KMeans
from sklearn import metrics
import pandas as pd
import shapely
import geopandas as gpd


In [18]:
import shapely
len(shapely.Point())

TypeError: object of type 'Point' has no len()

In [74]:
data = pd.read_csv(
    "/home/jyotiraditya/PycharmProjects/SiteReports/data/common_data/chennai/raw/location_poi_data.csv"
)[
    [
        "id",
        "name",
        "lat",
        "lng",
        "top_category",
        "category",
        "brand_name",
        "brand_id",
        "number_of_votes",
    ]
]
# mask = (
#     ((data["category"] == "restaurant") & (data["number_of_votes"] > 0) & (data["brand_id"] != "N_A"))|
#     ((data["category"] == "clothing_store") & (data["number_of_votes"] > 0) & (data["brand_id"] != "N_A"))|
#     ((data["category"] == "coffee_shop") & (data["number_of_votes"] > 10) & (data["brand_id"] != "NAA"))|
#     ((data["category"] == "shoe_store") & (data["number_of_votes"] > 0) & (data["brand_id"] != "N_A"))|
#     ((data["category"] == "jewelry_store") & (data["number_of_votes"] > 10) & (data["brand_id"] != "NAA"))|
#     ((data["category"] == "gym_fitness") & (data["number_of_votes"] > 10) & (data["brand_id"] != "NAA"))|
#     ((data["category"] == "bar_pub") & (data["number_of_votes"] > 10) & (data["brand_id"] != "NAA"))
    
# )
mask = (
    data["category"].isin(
        [
            "restaurant",
            "clothing_store",
            "coffee_shop",
            "shoe_store",
            "jewelry_store",
            "gym_fitness",
            "bar_pub",
        ]
    )
) & ((data["number_of_votes"] > 100) | (data["brand_id"] != "N_A"))
data = data[mask]
# data = data[data["city_name"] == "ahmedabad"]

In [92]:
data_gdf=gpd.GeoDataFrame(data,geometry=gpd.point_from_xy(data['lng'],data['lat']))

AttributeError: module 'geopandas' has no attribute 'point_from_xy'

In [79]:
def cluster_by_dbscan(df):
    data_array = np.radians(df[["lat", 'lng']].to_numpy())
    db = DBSCAN(eps=0.150/6371., min_samples=20, algorithm='ball_tree', metric='haversine').fit(data_array)
    df['label'] = db.labels_
    return df

def cluster_by_kmeans(df,num_clusters=60):
    data_array = np.radians(df[["lat", 'lng']].to_numpy())
    km=KMeans(n_clusters=num_clusters).fit(data_array)
    df['label'] = km.labels_
    return df

In [5]:
# from keplergl import KeplerGl
# map_2 = KeplerGl(height=1024, data={"data_1": df},  )
# map_2

In [80]:
df=data.copy()
print(df.shape)
df=cluster_by_dbscan(df)
df=df[df['label']!=-1]
print(df.shape)
# df=df.drop(columns=['label'])
# df=cluster_by_kmeans(df,50)

df.rename(columns={"label": "micro_market_clusters"}, inplace=True)

(11455, 9)
(2969, 10)


In [53]:
df.to_csv("mysuru micro market clusters.csv",index=False)

In [81]:
%%capture
df['geom_point'] = df.apply(lambda x: shapely.Point(x['lng'], x['lat']), axis=1)
df_ = df

clusters = df_.groupby("micro_market_clusters").agg(geom_point=("geom_point", list)).reset_index()
clusters.drop(clusters[clusters["micro_market_clusters"] ==-1].index, inplace=True)
# clusters.drop(clusters[clusters["micro_market_clusters"] < 5].index, inplace=True)

clusters["multipolygon"] = clusters["geom_point"].apply(lambda x: shapely.MultiPoint(x).convex_hull)
clusters['multipolygon']= clusters['multipolygon'].apply(lambda x: x.simplify(0.001 ).wkt)

clusters["num_points"] = clusters["geom_point"].apply(lambda x: len(x))
clusters.drop(clusters[clusters["num_points"] < 10].index, inplace=True)

clusters.drop(columns=["geom_point"], inplace=True)

clusters_gdf = gpd.GeoDataFrame(clusters, geometry=gpd.GeoSeries.from_wkt(clusters["multipolygon"]), crs="EPSG:4326")
clusters_gdf.to_crs(6933, inplace=True)
clusters_gdf["area"] = clusters_gdf["geometry"].area/1e6
clusters_gdf.to_crs(4326, inplace=True)

clusters_gdf["point_density"]  = clusters_gdf["num_points"]/clusters_gdf["area"]
clusters_gdf['centroid'] = clusters_gdf.centroid
clusters_gdf_ = clusters_gdf.drop(columns=["geometry"])

In [82]:
clusters_gdf_.to_csv("../data/common_data/chennai/high_streets.csv",index=False)

In [83]:
def split_large_polygon_by_density(polygon, points, base_polygon_area=0):
    if type(polygon) == str:
        polygon = shapely.from_wkt(polygon)
    try:
        from sklearn.cluster import KMeans
        polygon_area = polygon.area
        if polygon_area < base_polygon_area:
            return [polygon]
        else:
            num_splits = min( int(polygon_area / base_polygon_area), 4 )
            kmeans = KMeans(n_clusters=num_splits)
            kmeans.fit(points)

            cluster_labels = kmeans.labels_
            cluster_centers = kmeans.cluster_centers_
            points_by_poly = []
            for i in np.unique(cluster_labels):
                points_by_poly.append(points[cluster_labels == i])
            
            polygons = []
            for points in points_by_poly:
                polygon = shapely.MultiPoint([shapely.Point(i[0], i[1]) for i in points]).convex_hull.simplify(0.001)
                polygons.append(polygon)
            
            return polygons
    except Exception as e:
        print(e)
        return [polygon]

def geom_area_in_sqkm(geom):
    import contextlib
    import io
    import sys
    null_io = io.StringIO()
    with contextlib.redirect_stdout(null_io):
        if type(geom) == str:
            geom = shapely.from_wkt(geom)
        from pyproj import Geod
        geod = Geod(ellps="WGS84")
        area = geod.geometry_area_perimeter(geom)[0] / 10**6
    return -area if area < 0 else area



In [84]:
%%capture
clusters_gdf_['sub_polygons'] = clusters_gdf_.apply( lambda x: split_large_polygon_by_density(x['multipolygon'],df[df['micro_market_clusters'] == x['micro_market_clusters']][['lng', 'lat']].to_numpy(),0.000032), axis=1)

In [85]:
cluster_polygons_after_split = clusters_gdf_['sub_polygons'].explode()
clusters_gdf_['len_sub_polygons'] = clusters_gdf_['sub_polygons'].apply(lambda x: len(x))

clusters_gdf_ = pd.DataFrame(cluster_polygons_after_split).reset_index(drop=True)
clusters_gdf_ = clusters_gdf_.reset_index().rename(columns={"index": "micro_market_clusters_id", "sub_polygons": "polygon"})
clusters_gdf_['centroid'] = clusters_gdf_['polygon'].apply(lambda x: x.centroid)
clusters_gdf_['lat'] = clusters_gdf_['centroid'].apply(lambda x: x.y)
clusters_gdf_['lng'] = clusters_gdf_['centroid'].apply(lambda x: x.x)

In [ ]:
clusters_gdf_

In [86]:
clusters_gdf_.to_csv("../data/common_data/chennai/high_streets.csv",index=False)

In [88]:
def buffer_a_polygon_in_meters(polygon, meters):
    import pyproj
    from shapely.ops import transform
    
    wgs84 = pyproj.CRS('EPSG:4326')
    utm = pyproj.CRS('EPSG:32618')
    project_1 = pyproj.Transformer.from_crs(wgs84, utm, always_xy=True).transform
    project_2 = pyproj.Transformer.from_crs(utm, wgs84, always_xy=True).transform

    if type(polygon) == str:
        polygon = shapely.from_wkt(polygon)
    if meters > 0:
        polygon = transform(project_1, polygon)
        polygon = polygon.buffer(meters)
        polygon = transform(project_2, polygon)
    return polygon

def buffer_polygons(polygon,min,max, buffer_area_cutt_off):
    if type(polygon) == str:
        polygon = shapely.from_wkt(polygon)
    area = geom_area_in_sqkm(polygon)*1e6
    if area < buffer_area_cutt_off:
        amount_to_buffer = np.sqrt( (buffer_area_cutt_off - area) / (2*3.14)  )
        amount_to_buffer = np.clip(amount_to_buffer, min, max)
        polygon = buffer_a_polygon_in_meters(polygon, amount_to_buffer)
    return polygon


In [89]:
clusters_gdf_['polygon_updated'] = clusters_gdf_['polygon'].apply(lambda x: buffer_polygons(x, 0, 300,  0.26 * 1e6))

In [90]:
clusters_gdf_.to_csv("../data/common_data/chennai/high_streets.csv",index=False)